In [1]:
import math
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import time as t

In [2]:
import pandas as pd
import sys
sys.path.insert(1,"../Simulation_Pybullet/")
from KinovaGen3Class import KinovaGen3


In [3]:
#To improve the velocity, run it on the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device ', device)



#Create a experiment env
robot = KinovaGen3()

#Initially parameters of the urdf


print('observation space:', env.observation_space) #states, There is only 1 state constant
print('action space:', env.action_space) #parameters, number of parameters choose to tune, continuous

#t.sleep(10)


#Creation of a class to choose the parameters, the policy
class Agent(nn.Module):
    def __init__(self, env, h_size=16):
        super(Agent, self).__init__() #Means that this class heritage the init from nn.Module class
                                      # nn.Module it's the base class for all the neural net networks
                                      # https://pytorch.org/docs/stable/_modules/torch/nn/modules/module.html#Module
                                      # https://pytorch.org/docs/stable/nn.html
        
        self.env = env #Save the enviorment 
        
        #Dimensions constant = 1, due to the aren't inputs, hidden layer, parameters sizes
        self.s_size = env.observation_space.shape[0] #First layer 1 as there is only 1 state
        self.h_size = h_size #hidden layer
        self.a_size = env.action_space.shape[0] #Last layer number of parameters to tune, in tant per 1 to reach faster results
        
        # define layers
        self.fc1 = nn.Linear(self.s_size, self.h_size) # A linear layer that connect the states with the hidden layer
        self.fc2 = nn.Linear(self.h_size, self.a_size) # Hidden layer the from hidden layer to actions
        
    def set_weights(self, weights):
        s_size = self.s_size
        h_size = self.h_size
        a_size = self.a_size
        
        
        
       
        # Are linear layers so 
        # weight _w and b it's the bias.
        #https://medium.com/datathings/linear-layers-explained-in-a-simple-way-2319a9c2d1aa
        
        
        #The bias learns a constant value, independent of the input.
        
        # Learns that all the positive states need at least a bias constant + something dependant
        
        # a linear layer learns that the output ,it's the Activation Function ( input * pendent (the weight) + constant)
        
        # linear neuron output = input *w + b
        
        # separate the weights for each layer
        
        # so we are saying that (state1 * wl1 + bl1)*wl2 +bl2, so we belive it follows a 1st order equation * activation function
        
        
        fc1_end = (s_size*h_size)+h_size 
        #The first states * number of hidden layers are the weights of the first layer, each network has different weights for each state input
        fc1_W = torch.from_numpy(weights[:s_size*h_size].reshape(s_size, h_size))
        #From the previous end , the follwing hidden layer neurons number weights are the bias, each neuron has only 1 bias, doesn't depend on the state input
        fc1_b = torch.from_numpy(weights[s_size*h_size:fc1_end])
        
        #Every neuron has a weight for each action output
        fc2_W = torch.from_numpy(weights[fc1_end:fc1_end+(h_size*a_size)].reshape(h_size, a_size))
        fc2_b = torch.from_numpy(weights[fc1_end+(h_size*a_size):])
       
        # set the weights for each layer
        self.fc1.weight.data.copy_(fc1_W.view_as(self.fc1.weight.data))
        self.fc1.bias.data.copy_(fc1_b.view_as(self.fc1.bias.data))
        self.fc2.weight.data.copy_(fc2_W.view_as(self.fc2.weight.data))
        self.fc2.bias.data.copy_(fc2_b.view_as(self.fc2.bias.data))
    
    def get_weights_dim(self):
        #In reality its returning the weights + bias dimensions, the +1 its the bias
        return (self.s_size+1)*self.h_size + (self.h_size+1)*self.a_size
        
    def forward(self, x):
        #forward its Method to be able to pass the data as batches of data
        # It passes the data as Matrices adding the activation functions at the same time
        
        #They have activation functions to
        
        #https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6
        
        x = F.relu(self.fc1(x)) # Only possitive values pass
        #x = F.tanh(self.fc2(x))
        x = torch.tanh(self.fc2(x)) # Only from -1 to 1 values, as the action are 
                                    #go back , stay or go forward
                                    #it's perfect for the last layer
        return x.cpu().data
        
    def evaluate(self, weights, gamma=1.0, max_t=5000):
        # Obtain the accumulative reward from the actions selected by the neural net
        self.set_weights(weights)
        episode_return = 0.0
        state = self.env.reset()
        for t in range(max_t):
            state = torch.from_numpy(state).float().to(device)
            action = self.forward(state)
            state, reward, done, _ = self.env.step(action)
            episode_return += reward * math.pow(gamma, t)
            if done:
                break
        return episode_return
#End of class



Device  cpu
models/urdf/JACO3_URDF_V11.urdf


error: Not connected to physics server.

In [ ]:
print( env.observation_space.shape[0] ) #First layer number of states+
h_size=16
print( env.action_space.shape[0] ) #hidden layer

In [ ]:
agent = Agent(env).to(device) # Creation of a neural net in the device, in my case the GPU

In [ ]:
#Cross Entrophy Method, to choose the weights

# In my case where only 1 action,and that it's apply the parameters do another step doesn't change anything due to the state doesn't vary
# For this reason max_t and gama doesn't make sense, so I set them to max_t = 1 and gamma to 0
def cem(n_iterations=500, max_t=1, gamma=0.0, print_every=500*0.1, pop_size=50, elite_frac=0.2, sigma=0.5):
    """PyTorch implementation of the cross-entropy method.
        
    Params
    ======
        n_iterations (int): maximum number of training iterations
        max_t (int): maximum number of timesteps per episode
        gamma (float): discount rate
        print_every (int): how often to print average score (over last 100 episodes)
        pop_size (int): size of population at each iteration
        elite_frac (float): percentage of top performers to use in update
        sigma (float): standard deviation of additive noise
    """
    #Numbers of elements that you keep as the better ones
    n_elite=int(pop_size*elite_frac)
    
    #scores doble end queee , from iterations size * 0.1
    scores_deque = deque(maxlen=n_iterations*0.1)
    #intial scores empty
    scores = []
    
    #Initial best weights, are from 0 to 1, it's good to be small the weights, but they should be different from 0.
    # small to avoid overfiting , different from 0 to update them
    
    best_weight = sigma*np.random.randn(agent.get_weights_dim())

    #Each iteration, modify  + (from 0 to 1) the best weight randomly
    #Computes the reward with these weights
    #Sort the reward to get the best ones
    # Save the best weights
    # the Best weight it's the mean of the best one
    #compute the main reward of the main best rewards ones
    #this it's show to evalute how good its
    
    for i_iteration in range(1, n_iterations+1):
        
        #Generate new population weights, as a mutation of the best weight to test them
        weights_pop = [best_weight + (sigma*np.random.randn(agent.get_weights_dim())) for i in range(pop_size)]
        #Compute the parameters and obtain the rewards for each of them
        rewards = np.array([agent.evaluate(weights, gamma, max_t) for weights in weights_pop])
        
        #Sort the rewards to obtain the best ones
        elite_idxs = rewards.argsort()[-n_elite:]
        elite_weights = [weights_pop[i] for i in elite_idxs]
        
        #Set the best weight as the mean of the best ones 
        best_weight = np.array(elite_weights).mean(axis=0)
        
        #Get the reward with this new weight
        reward = agent.evaluate(best_weight, gamma=gamma)
        scores_deque.append(reward)
        scores.append(reward)
        
        #save the check point
        torch.save(agent.state_dict(), 'checkpoint.pth')
        
        if i_iteration % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_iteration, np.mean(scores_deque)))

        if np.mean(scores_deque)>=0.0:
            print('\nEnvironment solved in {:d} iterations!\tAverage Score: {:.2f}'.format(i_iteration-n_iterations*0.1, np.mean(scores_deque)))
            break
    return scores


In [ ]:
#Execute the cross entrophy method with default Values
#scores = cem()


#To don't ask the GPU as much reduce the pop_size, it's the amount of elemts try
scores = cem(pop_size=30)
# 
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
# load the weights from file
agent.load_state_dict(torch.load('checkpoint.pth'))

state = env.reset()
while True:
    state = torch.from_numpy(state).float().to(device)
    with torch.no_grad():
        action = agent(state)
    env.render()
    t.sleep(0.02)
    next_state, reward, done, _ = env.step(action)
    state = next_state
    if done:
        break

env.close()

In [ ]:
env.close()